# Basic Babamul example

In this notebook we'll read from Babamul streams to fetch alerts of interest
and save them locally,
e.g., to use in a machine learning pipeline.

In [ ]:
# First, load secrets from a local .env file if present
import dotenv

dotenv.load_dotenv()

In [ ]:
import os

import babamul


def alert_is_relevant(alert: babamul.ZtfAlert | babamul.LsstAlert) -> bool:
    """Determine if an alert is relevant for our purposes."""
    if isinstance(alert, babamul.ZtfAlert):
        # Example: Only care about alerts with mag < 18
        return (
            alert.magnitudes["ztf_r"] is not None
            and alert.magnitudes["ztf_r"] < 18.0
        )
    elif isinstance(alert, babamul.LsstAlert):
        # Example: Only care about alerts with mag < 20
        return (
            alert.magnitudes["lsst_r"] is not None
            and alert.magnitudes["lsst_r"] < 20.0
        )


alerts_dir = "data/alerts"
os.makedirs(alerts_dir, exist_ok=True)

alerts = []

with babamul.AlertConsumer(
    topics=babamul.topics.ALL_TOPICS, offset="latest"
) as consumer:
    for alert in consumer:
        if not isinstance(alert, (babamul.ZtfAlert, babamul.LsstAlert)):
            continue
        if alert_is_relevant(alert):
            print(f"Relevant alert: {alert.candid}")
            alerts.append(alert)
            # TODO: Save the alert to a local folder for later analysis

# TODO: Save alerts we like to a local folder for later analysis